# Intro to classes

This course assumes users have familarity with clasesses, however, if that's not the case, this section should help fill in the basics.

## Intro to a class

This is what a class looks like:

In [ ]:
class Simple:
    def function(self) -> None:
        print("hi!")

A class is a collection of objects, usually functions called "methods", that can produce instances. The instances usually also can contain objects, which are usually data values called "members". (Note on terminology: an instance of a class _is_ an object.) Following [PEP 8](https://www.python.org/dev/peps/pep-0008/), classes are named in CamelCase (though many classes in Python do not follow this; `int` is a class, for example).

Let's make an instance of our Simple class:

In [ ]:
simp = Simple()

We "call" the class to get an instance. We can then access a "method" on the instance:

In [ ]:
simp.function()

We did not store any state in the class, but usually you do. Let's look at a built-in class, `int`:

In [ ]:
my_int = int(3)

Since this is so common, there's a built in shortcut for this - we could have used `my_int = 3` directly - Python turns numbers into integers when it sees them. We can call methods, too:

In [ ]:
my_int.bit_length()

It takes 2 bits to be able to represent this integer. Python uses many more than that, but this is useful information about integers.

> Note: you cannot write `3.bit_length()`; due to the Python parser, this is invalid syntax. You can, however, do this with a float. `2.0.is_integer()` is valid, for example.

## Special methods

We can't go very far without writing a special method. Python has a lot of speical methods that have double underscores before and after the name - called "dunder methods". These customize all sorts of things about the class. Let's look at the most important one:

In [ ]:
class Simple:
    def __init__(self, msg: str):
        self.msg = msg

    def function(self) -> None:
        print(self.msg)

Now we finally can see a member! It's called `msg` - and it's stored on the instance, not the class. Let's try it out:

In [ ]:
simple_1 = Simple("I'm first")
simple_2 = Simple("I'm second")

In [ ]:
simple_1.function()
simple_2.function()

In [ ]:
print(simple_1.msg)
print(simple_2.msg)

Each instance stores a value of `msg` inside, while `function` is stored on the class.

In [ ]:
simple_1.msg == simple_2.msg

In [ ]:
simple_1.function == simple_2.function

Wait, what? Didn't I just tell you that `function` was stored on the class? Yes, it is - but accessing it this way is a shortcut - notice you didn't have to pass `self` in? So `simple_1.function(` is short for `Simple.function(simple_1,`. The object is added as the first parameter when you access a method on an instance, creating a "bound method".

Armed with our new knoledge, let's try calling the class methods by hand:

In [ ]:
Simple.function(simple_1)
Simple.function(simple_2)

> Aside: This is not at all how you'd normally call this, but this is useful later when reasoning about how other features work, and for a sneaky trick: _any object_ can be passed in, not just an instance of the class (was not true in Python 2). In this case, the object simply needs to have a `.msg` attribute to be used in `Simple.function`. If you are tempted to write a library that provides both free functions and methods to do the same thing, this means they can be one and the same, avoiding duplication for you and learning two APIs for your users.